# Activation functions

![](Image/Image7.jpg)

上圖的 neural networks 可以用以下程式碼建立 (底層的方法) ：

In [1]:
# import
import torch

# define the input layer
inputs_layer = torch.tensor([2., 1.])

# define the weights between the inputs and the first hidden layer
weight1 = torch.tensor([[0.45, 0.32], [-0.12, 0.29]])

# define the first layer
first_layer = torch.matmul(inputs_layer, weight1)

# define the weights between the first layer and the output layer
weight2 = torch.tensor([[0.48, -0.12], [0.64, 0.91]])

# define the outputs
outputs_layer = torch.matmul(first_layer, weight2)

# print the results
print(outputs_layer)

tensor([0.9696, 0.7527])


**測試：將過程中的 weights 矩陣相乘看看結果是否不一樣**

In [4]:
# 將過程中的 weights 矩陣相乘
weights = torch.matmul(weight1, weight2)
print(weights)

# 計算輸出
outputs_layer2 = torch.matmul(inputs_layer, weights)

# print the results
print(outputs_layer2)

tensor([[0.4208, 0.2372],
        [0.1280, 0.2783]])
tensor([0.9696, 0.7527])


### 線性關係
由於矩陣相乘就代表了 linear transformation，結合在一起也代表了同時做了多個 transformation，因此上面兩者的結果是一樣的。這也代表我們實際上可以只用一層 input layer 和一層 output layer 以及連接兩者之間的 weights 就可以達到相同的效果。事實上，任何 neural network 都可以被簡化成這種沒有 hidden layer 的 network。

### 非線性關係
基本上，所有 neural networks 都可以被簡化成沒有 hidden layers 的 network。然而，這樣代表輸出層和輸入層的關係為線性關係。也就是說，這個模型只能解釋資料中的線性關係，例如：二維平面上的一條線、三維平面上的一個平面 (下圖左)。事實上，資料之間的關係不一定是線性的，有些資料間擁有非常複雜的關係，例如下圖右。因此，我們要在模型中加入一些非線性的函數，也就是 activation functions

![](Image/Image8.jpg)

## 常用的 activation functions

![](Image/Image9.jpg)

以及 Softmax，可以用在 multiclass classification problems 的 output layer。

當加入這些 activation functions 之後，上面例子的結果就不會相同了。

### Implementing ReLU (Rectified Linear Unit) in PyTorch

In [5]:
# import
import torch.nn as nn

# define a ReLU function
relu = nn.ReLU()

# define a tensor
t1 = torch.tensor([2., -4.])

# calculate the output of ReLU function
print(relu(t1))

# define another tensor
t2 = torch.tensor([[2., -4.], [1.2, 0.]])

# calculate the output of ReLU function
print(relu(t2))

tensor([2., 0.])
tensor([[2.0000, 0.0000],
        [1.2000, 0.0000]])


### Implementing neural networks with activation functions (from the previous example)

In [6]:
# import
import torch
import torch.nn as nn

# define a ReLU function
relu = nn.ReLU()

# define the input layer
inputs_layer = torch.tensor([2., 1.])

# define the weights between the inputs and the first hidden layer
weight1 = torch.tensor([[0.45, 0.32], [-0.12, 0.29]])

# define the first layer
first_layer = relu(torch.matmul(inputs_layer, weight1))

# define the weights between the first layer and the output layer
weight2 = torch.tensor([[0.48, -0.12], [0.64, 0.91]])

# define the outputs
outputs_layer = relu(torch.matmul(first_layer, weight2))

# print the results
print(outputs_layer)

tensor([0.9696, 0.7527])


# Loss functions

訓練 neural networks 的過程：

1. 隨機給 weights 一些初始值
2. 將所有 input 的資料帶入，利用 forward propagation 計算輸出值
3. 評估輸出值的表現
4. 更新 weights 並重新帶入所有相同的 input 資料

過程中，評估輸出值表現的方法，就是利用 loss function 來量化模型的好壞 (例如 MSE、MAE 等等)

而更新 weights 就必須計算該 loss function 的 gradient (因此 loss function 必須是 differentiable)，然後乘上自定義的 learning rate ，作為 weight 的變化方向與變化量

---

一般來說，regression model 常用的 loss function 是 mean square error (MSE)，而 multiclass classification model 常用 Softmax cross-entropy loss

## Softmax Cross-entropy Loss

$$ P(Y = k|X = xi) = \frac{exp(s)*k}{ \sum_{j}\ exp(s)*j } $$

這個公式負責將 multi-class classification neural networks 輸出的數字轉換成機率。計算方法如下：

假設現在有一張貓的照片，而我想要用一個 neural network 來分類這張照片是貓、車子、還是青蛙。所以我將這張照片轉換成 vector 餵給 network (有三個 output nodes)。我會得到三個數字：

![](Image/Image10.jpg)

由於是 multiclass classification problem，因此不能用 MSE 來計算模型表現，也不能用準確率來評估 (因為準確率不是一個可微分的函數)，因此我們要將輸出的數字轉換成機率分布。如下圖：

![](Image/Image11.jpg)

由於機率皆大於 0，因此第一步是將數字都帶入 exponential function，接下來 normalize 就可以得到各類別的機率。有了機率，就可以計算 cross-entropy loss。

$$ CrossEntropy Loss = -ln(正確的類別的機率) $$

假設我的 network 給出貓的機率為 1 (完全認出是貓)，則 $ CrossEntropy Loss = -ln(1) = 0 $，很合理。假設我的 network 給出貓的機率接近 0 (完全沒認出來)，則 $ CrossEntropy Loss = -ln(0) = 無限大 $，代表 loss 超大，必須繼續縮小。

## Implement Cross-entropy loss in PyTorch

In [9]:
# import
import torch
import torch.nn as nn

# define an output scores
output_scores = torch.tensor([[3.2, 5.1, -1.7]])

# Specify the currect class number
true_class = torch.tensor([0])    # 0 代表第一個 element 是正確的類別

# define a cross-entropy loss object
criterion = nn.CrossEntropyLoss()

# 計算 loss
loss = criterion(output_scores, true_class)

# print
print(loss)

tensor(2.0404)


**給定另一組 output scores**

In [14]:
# define an output scores
output_scores2 = torch.tensor([[10.2, 5.1, -1.7]])

# Specify the currect class number
true_class = torch.tensor([0])    # 0 代表第一個 element 是正確的類別

# define a cross-entropy loss object
criterion = nn.CrossEntropyLoss()

# 計算 loss
loss2 = criterion(output_scores2, true_class)

# print
print(loss2)

tensor(0.0061)


可以發現 loss 少多了

# Preparing a dataset in PyTorch

這裡會使用 MNIST、CIFAR-10 這兩個 dataset 來示範 (**皆直接包含於 PyTorch 這個套件中**)。

MNIST 是 computer vision 中最具代表性的資料集，包含 70000 個手寫的 digits (每個數字都有 6000 筆訓練資料以及 1000 筆測試資料)。手寫 digits 的照片都是灰階的，而且解析度為 28 * 28。

CIFAR-10 是一些自然景觀的彩色照片，總共有 60000 張照片 (包含 airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck 等 10 個類別)，其中 50000 張照片是訓練集，剩下 10000 張照片是測試集。解析度為 32 * 32，且因為是彩色，所以有 RGB 三個維度。

### 先用 CIFAR-10 這個資料集

In [52]:
# import
import torch
import torchvision    # a package which deals with datasets and pre-trained neural networks
import torch.utils.data
import torchvision.transforms as transforms

In [53]:
# define a transformation of images to torch tensors using torchvision transforms 
# 下載資料時就可以直接用這個定義好的 transform 物件來轉換資料
transformCIFAR = transforms.Compose(
    [transforms.ToTensor(),    # define a transformation of images object
    transforms.Normalize((0.4914, 0.48216, 0.44653),    # R, G, B 的平均 (pre-computed) 用來將數據標準化
                        (0.24703, 0.24349, 0.26159))]    # R, G, B 的標準差 (pre-computed) 用來將數據標準化
)

In [54]:
# download the CIFAR-10 datasets
training_setCIFAR = torchvision.datasets.CIFAR10(root = "./Datasets/CIFAR-10", train = True, download = True, transform = transformCIFAR)
testing_setCIFAR = torchvision.datasets.CIFAR10(root = "./Datasets/CIFAR-10", train = False, download = True, transform = transformCIFAR)

Files already downloaded and verified
Files already downloaded and verified


In [55]:
# load the CIFAR-10 datasets
trainloaderCIFAR = torch.utils.data.DataLoader(training_setCIFAR, batch_size = 32, shuffle = True, num_workers = 4)
testloaderCIFAR = torch.utils.data.DataLoader(testing_setCIFAR, batch_size = 32, shuffle = False, num_workers = 4)

由於 datasets 太大，因此我們設定 load 的 **batch size** 是 32，代表一次只用 32 筆資料來更新 weights and bias。**shuffle** 代表是否打亂資料的順序，為了增加 randomness，我們設定為 True。**num_workers** 代表訓練過程要使用多少個 processors。

**Inspect the data loaders**

In [56]:
# inspect the shape of the training dataset
print(trainloaderCIFAR.dataset.train_data.shape)

# inspect the shape of the testing dataset
print(testloaderCIFAR.dataset.test_data.shape)

AttributeError: 'CIFAR10' object has no attribute 'train_data'

In [57]:
# inspect batch size
print(testloaderCIFAR.batch_size)

32


In [58]:
# inspect the type of random sampler
print(trainloaderCIFAR.sampler)

The CIFAR-10 dataset is well prepared to be used in PyTorch

### 再用 MNIST 這個資料集

In [59]:
# Transform the data to torch tensors and normalize it 
transformMNIST = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307), ((0.3081)))])   # 由於 MNIST 是灰階，因此沒有像 RGB 一樣三個平均和三個標準差，這裡平均為 0.1307，標準差為 0.3081

# Prepare training set and testing set
trainsetMNIST = torchvision.datasets.MNIST(root = "./Datasets/MNIST", train=True, 
                                      download=True, transform=transformMNIST)
testsetMNIST = torchvision.datasets.MNIST(root = "./Datasets/MNIST", train=False, 
                                     download=True, transform=transformMNIST)

# Prepare training loader and testing loader
trainloaderMNIST = torch.utils.data.DataLoader(trainsetMNIST, batch_size = 32,
                                          shuffle = True, num_workers=8)
testloaderMNIST = torch.utils.data.DataLoader(testsetMNIST, batch_size = 32,
                                         shuffle = False, num_workers=8) 

**Inspect the dataloaders**

In [60]:
# Compute the shape of the training set and testing set
trainset_shapeMNIST = trainloaderMNIST.dataset.train_data.shape
testset_shapeMNIST = testloaderMNIST.dataset.test_data.shape

# Print the computed shapes
print(trainset_shapeMNIST, testset_shapeMNIST)

# Compute the size of the minibatch for training set and testing set
trainset_batchsizeMNIST = trainloaderMNIST.batch_size
testset_batchsizeMNIST = testloaderMNIST.batch_size

# Print sizes of the minibatch
print(trainset_batchsizeMNIST, testset_batchsizeMNIST)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])
32 32


C:\Users\TangKuoLin\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
C:\Users\TangKuoLin\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


The MNIST dataset is well prepared to be used in PyTorch

# Training neural networks

## 先用 CIFAR-10 這個資料集

### 建立模型 (設計模型的 class)

In [66]:
# import
import torch.nn as nn
import torch.nn.functional as F    # Provide a more functional way than the nn module (我們用這個 module 提供的 relu 函數當作 activation function)

# 寫好 neural network 的 class
class CIFAR_Net(nn.Module):
    # constructor 中我們會透過 specify weights 來定義模型中各個 layers 的 nodes 的數量 
    def __init__(self):
        super(CIFAR_Net, self).__init__()
        # fully connected layers (dense layers) 在 PyTorch 中稱為 nn.Linear(目前layer的nodes數, 下一層layer的nodes數)
        self.fc1 = nn.Linear(32 * 32 * 3, 500)    # 由於每張照片都是 32 * 32 且有 RGB 三原色，因此一張照片總共有 32 * 32 * 3 個資料點
        self.fc2 = nn.Linear(500, 10)    # 500 代表 hidden layer 有 500 個 nodes， 10 代表輸出層共有 10 個 nodes (10 個類別)
        
    def forward(self, x):    # inputs 為 x
        # apply all weights to the inputs
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Training

In [67]:
# import
import torch.optim as optim    # 這個套件包含許多最佳化的方法

# 建立模型物件
net = CIFAR_Net()

# 建立 cross entropy loss function 的物件
criterion = nn.CrossEntropyLoss()

# 設定 optimizer
optimizer = optim.Adam(net.parameters(), lr = 3e-4)

for epoch in range(10):    # 所有的資料要被訓練10次 (10 個 epochs)
    for i, data in enumerate(trainloaderCIFAR, start = 0):    # 每個 batch 之後都會更新 weights (i 為 batch number, data 為每個 batch 的資料集)
        # get the inputs
        inputs, labels = data
        inputs = inputs.view(-1, 32 * 32 * 3)    # 將所有的資料點變成一維向量
        
        # 將 optimizer 的 gradient 歸零，以避免迴圈前一次的 gradient 的影響。每一次更新 weights 的 gradients 都是重新開始算的
        optimizer.zero_grad()
        
        # Forward propagation (計算 output)
        outputs = net(inputs)
        
        # optimisation
        loss = criterion(outputs, labels)
        
        # Backpropagation
        loss.backward()    # 計算 loss 這個函數中各個 weights 的 gradients
        
        # 利用計算得到的 gradients 更新 weights
        optimizer.step()

### Testing

In [68]:
correct, total = 0, 0
predictions = []

# 將 net 這個 Net 的物件設定為 evaluation mode
net.eval()

for i, data in enumerate(testloaderCIFAR, start = 0):
    # get the testing data
    inputs, labels = data
    inputs = inputs.view(-1, 32 * 32 * 3)
    
    # 預測結果 (計算 output scores for each class)
    outputs = net(inputs)
    
    # 將 output scores 轉換成類別 (誰的 output score 最大就屬於哪個類別)
    _, predicted = torch.max(outputs.data, 1)
    
    # 將預測的類別轉換成 list
    predictions.append(outputs)
    
    # 計算資料的總數
    total += labels.size(0)
    
    # 計算預測正確的數量
    correct += (predicted == labels).sum().item()
    
# 印出預測正確率    
print("The CIFAR-10  testing set accuracy of the network if: %d %%" % (100 * correct / total))

The CIFAR-10  testing set accuracy of the network if: 51 %


預測正確率為 53%

## 再用 MNIST 這個資料集

### 建立模型 (設計模型的 class)

In [69]:
# Define the class Net
class MNIST_Net(nn.Module):
    def __init__(self):    
    	# Define all the parameters of the net
        super(MNIST_Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28 * 1, 200)    # 黑白照片，解析度 28 * 28
        self.fc2 = nn.Linear(200, 10)    # hidden layer 200 nodes，output layer 10 nodes (0 ~ 9 個數字)

    def forward(self, x):   
    	# Do the forward pass
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Training

In [70]:
# Instantiate the Adam optimizer and Cross-Entropy loss function
model = MNIST_Net()   
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
  
for batch_idx, data in enumerate(trainloaderMNIST):
    indenpendent_variables = data[0]    
    labels = data[1]
    indenpendent_variables = indenpendent_variables.view(-1, 28 * 28)
    
    # reset gradients in optimizer
    optimizer.zero_grad()

    # Complete a forward pass
    output = model(indenpendent_variables)

    # Compute the loss, gradients and change the weights
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()

### Testing

In [71]:
# Set the model in eval mode
model.eval()

for batch_idx, data in enumerate(testloaderMNIST, 0):
    independent_variables, labels = data
    
    # Put each image into a vector
    independent_variables = independent_variables.view(-1, 28 * 28 * 1)
    
    # Do the forward pass and get the predictions
    outputs = model(independent_variables)
    
    # 將 predicted scores 轉換成類別
    _, outputs = torch.max(outputs.data, 1)
    
    # 計算資料總數以及預測正確的資料數
    total += labels.size(0)
    correct += (outputs == labels).sum().item()
    
print('The testing set accuracy of the network is: %d %%' % (100 * correct / total))

The testing set accuracy of the network is: 73 %


預測正確率為 73%